In [1]:
from sklearn import datasets, metrics,linear_model,preprocessing,model_selection
import numpy as np
import random


In [20]:
class MyLogisticRegression:
    
    def __init__(self):
        self.intercepts = None
        
    def __calcProbability(self,iv,params):
        return(1/(1+np.exp(-np.dot(params.T,iv))))

    def predict(self,test_X,prob_threshold=0.5):
        #Threshold not required for Multiclass Classification
        if(self.intercepts==None or len(self.intercepts)==0 ):
            raise Exception('Model not fitted.')
            
        test_X=np.column_stack((np.ones(len(test_X)),test_X))
        final_prediction=[]
        if len(self.intercepts)>1:#Multiclass classification
            probsList=[]      
            #Predict the probability using each model
            class_labels=[]
            for cl,params in self.intercepts.items():                  
                probs=np.apply_along_axis(self.__calcProbability,1,test_X,params)#using each model predict the probability
                class_labels.append(cl)#keep list of all the class labels
                probsList.append(probs)
            #Check which model gave highest probability to class 1 and use the corresponding class label.
            final_prediction=np.apply_along_axis(lambda x:class_labels[np.argmax(x)],0,probsList)
            
        else:# Binary classification
            probs=np.apply_along_axis(self.__calcProbability,1,test_X,self.intercepts.get(1))  
            probs[probs>=prob_threshold]=1
            probs[probs<prob_threshold]=0
            final_prediction=probs
        return(final_prediction)

    def __gradientDescent(self,x,y,no_iter,learning_rate,stochastic,lambda_val):
        N=len(x)
        x_actual=np.column_stack((np.ones(N),x))#add new column having values 1 for y intercept parameter
        y_actual=y
        params=np.zeros(x_actual.shape[1]) #initialize the parameters to 0. i.e B0=B1=B2=0
        for i in range(no_iter):
            if(stochastic):
                random_num=random.sample(range(0, N), 1)            
                x_temp=x_actual[random_num,:]
                y_temp=y_actual[random_num]        
            gradients=np.zeros(x_temp.shape[1])#initialize gradients    
            #predict probability for each sample. apply function along each row
            all_pred=np.apply_along_axis(self.__calcProbability,1,x_temp,params)
            for i in range(x_temp.shape[1]):
                #calculate gradient of each feature                     
                grad=sum((all_pred-y_temp)*x_temp[:,i])+ 2*lambda_val*params[i]
                gradients[i]=grad
            
            #recalculate new parameters
            params=params-(learning_rate*gradients)
        return params

    def fit(self,train_X, train_y, alpha=0.01, num_iterations=10000, lambda_reg=0.0001,stochastic=True):
        
        classes=np.unique(train_y)
        models={}
        if(len(classes)>2):            
            #One vs Rest Approach.
            #A model will be created for each class. with 1 label being that of target class
            #and all the other classes will be class 0
            for cl in classes:
                ty=train_y.copy() 
                ty[ty!=cl]=0
                ty[ty==cl]=1
                #Train each of the model using gradient descent
                #store the trained coefficients in a dictionary with the key being the actual label of class 1
                params=self.__gradientDescent(train_X,ty,num_iterations,alpha,stochastic,lambda_reg)
                models.update({cl:params})
               
        else:
            params=self.__gradientDescent(train_X,train_y,num_iterations,alpha,stochastic,lambda_reg)
            models.update({1:params})
        self.intercepts=models


In [2]:
'''LOAD DATASET'''
covtype=datasets.fetch_covtype()
x = covtype.data
y = covtype.target


In [3]:
'''split into train and test'''

x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y,random_state = 18,test_size=0.3)
x_train=preprocessing.scale(x_train)
x_test=preprocessing.scale(x_test)


<b>As I have used Stochastic Gradient Descent, the number of iterations is relatively large. Also as the model was underfitting,
    I have kept the regularization parameter small. The learning rate has been set to 0.01 after trying different values</b>
    

In [27]:
'''Test Custom Logistic Regression'''
model=MyLogisticRegression()            
#Tuned Parameters. As I have used stochastic GD, the number of iterations is high
model.fit(x_train,y_train,alpha=0.01,num_iterations=30000,lambda_reg=0.001)
y_pred=model.predict(x_train)
print('ACCURACY ON TRAIN',metrics.accuracy_score(y_pred,y_train))
y_pred_test=model.predict(x_test)    
print('ACCURACY ON TEST',metrics.accuracy_score(y_pred_test,y_test))


ACCURACY ON TRAIN 0.704753779124089
ACCURACY ON TEST 0.7063463833302736


In [5]:
'''Test using Sklearn Logistic Regression'''
sklearn_logisticmodel=linear_model.LogisticRegression()#Default is ridge in sklearn. 
sklearn_logisticmodel.fit(x_train,y_train)    
sklearn_pred_train=sklearn_logisticmodel.predict(x_train)
print('ACCURACY ON TRAIN',metrics.accuracy_score(sklearn_pred_train,y_train))
sklearn_pred_test=sklearn_logisticmodel.predict(x_test)    
print('ACCURACY ON TEST',metrics.accuracy_score(sklearn_pred_test,y_test))



ACCURACY ON TRAIN 0.7148347217168091
ACCURACY ON TEST 0.7156404901780796


In [6]:
sklearn_logisticmodel.get_params# we can increase the C value to reduce the regularization

<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)>

<b>Changing the solver to Stochastic Average Gradient Descent and using multinomial instead of ovr</b>

In [5]:
'''Tuned SKLEARN Logistic Regression'''
sklearn_logisticmodel2=linear_model.LogisticRegression(multi_class='multinomial',solver='saga',C=100)
sklearn_logisticmodel2.fit(x_train,y_train)    
sklearn_pred_train2=sklearn_logisticmodel2.predict(x_train)
print('ACCURACY ON TRAIN',metrics.accuracy_score(sklearn_pred_train2,y_train))
sklearn_pred_test2=sklearn_logisticmodel2.predict(x_test)    
print('ACCURACY ON TEST',metrics.accuracy_score(sklearn_pred_test2,y_test))


C:\Users\gudea\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


ACCURACY ON TRAIN 0.7243919470480049
ACCURACY ON TEST 0.7242633559757665


<b> We can conclude that the results of custom logistic regression model is quite similar to that of Sklearn implementation. We
    can further optimize it by using parallel processing and softmax instead of ovr.</b>